# Importanto bibliotecas e Definindo diretórios

In [ ]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report

data_dir = f"./data"
#corel-1k
dataset_dir = f"{data_dir}/dataset"
images_dir = f"{dataset_dir}/images"
#caltech256
dataset_1_dir = f"{data_dir}/dataset_1"
images_1_dir = f"{dataset_1_dir}/images"
#cifar100
dataset_2_dir = f"{data_dir}/dataset_2"
images_2_dir = f"{dataset_2_dir}/images"

# Funções p/ escala de cinza

## Geração de histogramas

In [ ]:
def obter_vizinhos(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [ ]:
def aplica_regras(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [ ]:
def percorre_imagem_aplicando_regras(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [ ]:
def gera_histogramas(imagem_cinza):
    #O objetivo dessa função é criar as matrizes de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem_cinza)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

In [ ]:
from tqdm import tqdm
def gerador_histogramas(dataset_dir, escala):
    #Funcao principal: faz a chamada das funções acima e salva os histogramas

    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_grayscale"

    # Cria as pastas para salvar os histogramas mantendo o padrão das classes do dataset
    os.makedirs(histograms_dir, exist_ok=True)
    classes = [conteudo_item for conteudo_item in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, conteudo_item))]
    for classe in classes:
        dir_pastas = os.path.join(histograms_dir, classe)
        os.makedirs(dir_pastas, exist_ok=True)


    #Esse loop pega a imagem, gera seus histogramas e salva com base no nome da classe e da imagem
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        classe = f"{classe}"
        
        lista_arquivos = os.listdir(dir_classe)
        num_arquivos = len(lista_arquivos)
        
        progress_bar = tqdm(total=num_arquivos, desc= classe)
        
        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"

            imagem_cinza = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE) #Lendo em escala de cinza
            imagem_cinza = cv2.resize(imagem_cinza, (escala,escala)) #Padronizando tamanho das imagens
            
            hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = gera_histogramas(imagem_cinza)

            img_name = imagem.split(".")[0]
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_phi_vivos.pkl")
            joblib.dump(hist_phi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_phi_mortos.pkl")
            joblib.dump(hist_phi_mortos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_psi_vivos.pkl")
            joblib.dump(hist_psi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_psi_mortos.pkl")
            joblib.dump(hist_psi_mortos, file_path)
            progress_bar.update(1)

        progress_bar.close()

    print("Progresso concluído")

## Preparação de dados

In [ ]:
def carrega_hist(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #./data/dataset/images/classe/imagem.jpg
    imagem_path = imagem_path.split("/")
    dataset, classe, imagem = imagem_path[2], imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    histogramas_dir = f"./data/{dataset}/histograms_grayscale"

    try:
        #Buscando os histogramas no diretorio
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print(imagem_path)
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [ ]:
def retorna_combinacao(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist(imagem_path)
    if combinacao == 0:
        return hist_phi_vivos
    elif combinacao == 1:
        return hist_phi_mortos
    elif combinacao == 2:
        return hist_psi_vivos
    elif combinacao == 3:
        return hist_psi_mortos
    elif combinacao == 4:
        return np.concatenate([hist_phi_vivos, hist_phi_mortos]) #histogramas phi
    elif combinacao == 5:
        return np.concatenate([hist_psi_vivos, hist_psi_mortos]) #histogramas psi
    elif combinacao == 6:
        return np.concatenate([hist_phi_vivos, hist_psi_vivos]) #histogramas vivos
    elif combinacao == 7:
        return np.concatenate([hist_phi_mortos, hist_psi_mortos]) #histogramas mortos
    elif combinacao == 8:
        return np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                               np.concatenate([hist_psi_vivos, hist_psi_mortos])])
        #histogramas phi e psi combinados

In [ ]:
# Prepara os dados para treinar o modelo
def retorna_dados(images_dir):
    histogramas = []
    rotulos = []
    
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        print(classe)
        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"
    
            #Pode alterar a combinacao
            combinacao = retorna_combinacao(imagem_path, 8)
    
            #Lista dos histogramas
            histogramas.append(combinacao)
            #lista da classe assossiada ao histograma
            rotulos.append(classe)
    
    print("Progresso concluído")
    return histogramas, rotulos

# Funções p/ RGB

## Geração de histogramas

In [ ]:
def obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [ ]:
def aplica_regras_rgb(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [ ]:
def percorre_imagem_aplicando_regras_rgb(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras_rgb(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [ ]:
def gera_histogramas_rgb(imagem):
    #O objetivo dessa função é criar as matrizes de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

In [ ]:
from tqdm import tqdm
def gerador_histogramas_RGB(dataset_dir, escala):
    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_rgb"

    #Criando diretorios das classes
    os.makedirs(histograms_dir, exist_ok=True)
    classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    for classe in classes:
        os.makedirs(os.path.join(histograms_dir, classe), exist_ok=True)

    for classe in classes:
        dir_classe = os.path.join(images_dir, classe)

        lista_arquivos = os.listdir(dir_classe)
        num_arquivos = len(lista_arquivos)
        progress_bar = tqdm(total=num_arquivos, desc= classe)
        
        for imagem in os.listdir(dir_classe):
            imagem_path = os.path.join(dir_classe, imagem)
            imagem_rgb = cv2.imread(imagem_path, cv2.IMREAD_COLOR) #Lendo a imagem em BGR
            imagem_rgb = cv2.resize(imagem_rgb, (escala, escala))

            #Gera os histogramas para cada canal da imagem
            hist_phi_vivos_b, hist_phi_mortos_b, hist_psi_vivos_b, hist_psi_mortos_b = gera_histogramas_rgb(imagem_rgb[:, :, 0])
            hist_phi_vivos_g, hist_phi_mortos_g, hist_psi_vivos_g, hist_psi_mortos_g = gera_histogramas_rgb(imagem_rgb[:, :, 1])
            hist_phi_vivos_r, hist_phi_mortos_r, hist_psi_vivos_r, hist_psi_mortos_r = gera_histogramas_rgb(imagem_rgb[:, :, 2])

            #Concatena os histogramas dos 3 canais para cada uma das 4 situações após o jogo da vida (vivos->vivos, vivos->mortos, etc.)
            hist_phi_vivos = np.concatenate([np.concatenate([hist_phi_vivos_b,hist_phi_vivos_g]),hist_phi_vivos_r])
            hist_phi_mortos = np.concatenate([np.concatenate([hist_phi_mortos_b,hist_phi_mortos_g]),hist_phi_mortos_r])
            hist_psi_vivos = np.concatenate([np.concatenate([hist_psi_vivos_b,hist_psi_vivos_g]),hist_psi_vivos_r])
            hist_psi_mortos = np.concatenate([np.concatenate([hist_psi_mortos_b,hist_psi_mortos_g]),hist_psi_mortos_r])

            #salvando os histogramas
            index = os.path.splitext(os.path.basename(imagem_path))[0]
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_vivos_bgr.pkl")
            joblib.dump(hist_phi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_mortos_bgr.pkl")
            joblib.dump(hist_phi_mortos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_vivos_bgr.pkl")
            joblib.dump(hist_psi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_mortos_bgr.pkl")
            joblib.dump(hist_psi_mortos, file_path)
            progress_bar.update(1)

                
        progress_bar.close()


    print("Progresso concluído")

## Preparação de dados

In [ ]:
def carrega_hist_rgb(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #./data/dataset/images/classe/imagem.jpg
    imagem_path = imagem_path.split("/")
    dataset, classe, imagem = imagem_path[2], imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    histogramas_dir = f"./data/{dataset}/histograms_rgb"
    ##print(imagem_path)
    ##print(histogramas_dir)
    
    try:
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos_bgr.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos_bgr.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos_bgr.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos_bgr.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [ ]:
def retorna_combinacao_rgb(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    
    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist_rgb(imagem_path)
    
    switch = {
        0: hist_phi_vivos,
        1: hist_phi_mortos,
        2: hist_psi_vivos,
        3: hist_psi_mortos,
        4: np.concatenate([hist_phi_vivos, hist_phi_mortos]),   #histogramas phi
        5: np.concatenate([hist_psi_vivos, hist_psi_mortos]),   #histogramas psi
        6: np.concatenate([hist_phi_vivos, hist_psi_vivos]),    #histogramas vivos
        7: np.concatenate([hist_phi_mortos, hist_psi_mortos]),  #histogramas mortos
        8: np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                           np.concatenate([hist_psi_vivos, hist_psi_mortos])])
    }
    
    return switch.get(combinacao, None)

In [ ]:
# Prepara os dados para treinar o modelo
def retorna_dados_rgb(images_dir): 
    histogramas_rgb = []
    rotulos_rgb = []
    
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        print(classe)
        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"
    
            #Pode alterar a combinacao 
            combinacao = retorna_combinacao_rgb(imagem_path, 8)
    
            #Lista dos histogramas
            histogramas_rgb.append(combinacao)
            #lista da classe assossiada ao histograma
            rotulos_rgb.append(classe)
    
    print("Progresso concluído")
    return histogramas_rgb, rotulos_rgb

# Funções p/ modelos

In [ ]:
def gaussian(X_train, y_train, X_test, y_test):
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    model1 = gnb.fit(X_train, y_train)
    aux = gnb.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def logistic_regr(X_train, y_train, X_test, y_test):
    # Logistic Regression
    logreg = LogisticRegression(max_iter = 1000)
    model2 = logreg.fit(X_train, y_train)
    aux = logreg.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def decision_tree(X_train, y_train, X_test, y_test):
    # Decision Tree
    dectree = DecisionTreeClassifier()
    model3 = dectree.fit(X_train, y_train)
    aux = dectree.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def knn(X_train, y_train, X_test, y_test):
    # K-Nearest Neighbors
    knn = KNeighborsClassifier(n_neighbors = 3)
    model4 = knn.fit(X_train, y_train)
    aux = knn.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def lda(X_train, y_train, X_test, y_test):
    # Linear Discriminant Analysis
    lda = LinearDiscriminantAnalysis()
    model5 = lda.fit(X_train, y_train)
    aux = lda.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def svm(X_train, y_train, X_test, y_test):
    # Support Vector Machine
    svm = SVC()
    model6 = svm.fit(X_train, y_train)
    aux = svm.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def random_forest(X_train, y_train, X_test, y_test):
    # RandomForest
    rf = RandomForestClassifier()
    model7 = rf.fit(X_train, y_train)
    aux = rf.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def neural_net(X_train, y_train, X_test, y_test):
    # Neural Net
    nnet = MLPClassifier(alpha=1)
    model8 = nnet.fit(X_train, y_train)
    aux = nnet.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

# Dataset Corel 1k

## Em escala de cinza:

### Gerando histogramas

In [ ]:
#gerador_histogramas(dataset_dir, 128)

### Testando modelos de aprendizado de máquina combinando histogramas

In [ ]:
histogramas, rotulos = retorna_dados(images_dir)
X = np.array(histogramas)
y = np.array(rotulos)

#Dividindo treino e teste na respectiva proporção (0.8 / 0.2), stratify = y mantém uma proporção entre os dados de teste de cada classe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em RGB:

### Gerando histogramas

In [ ]:
gerador_histogramas_RGB(dataset_dir, 128)

### Testando modelos de aprendizado

In [ ]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

# Dataset_1 Caltech256

## Em RBG:


### Gerando histogramas

In [ ]:
#gerador_histogramas_RGB(dataset_1_dir, 128)

### Testando modelos de aprendizado 

In [96]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_1_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

001.ak47
002.american-flag
003.backpack
004.baseball-bat
005.baseball-glove
006.basketball-hoop
007.bat
008.bathtub
009.bear
010.beer-mug
011.billiards
012.binoculars
013.birdbath
014.blimp
015.bonsai-101
016.boom-box
017.bowling-ball
018.bowling-pin
019.boxing-glove
020.brain-101
021.breadmaker
022.buddha-101
023.bulldozer
024.butterfly
025.cactus
026.cake
027.calculator
028.camel
029.cannon
030.canoe
031.car-tire
032.cartman
033.cd
034.centipede
035.cereal-box
036.chandelier-101
037.chess-board
038.chimp
039.chopsticks
040.cockroach
041.coffee-mug
042.coffin
043.coin
044.comet
045.computer-keyboard
046.computer-monitor
047.computer-mouse
048.conch
049.cormorant
050.covered-wagon
051.cowboy-hat
052.crab-101
053.desk-globe
054.diamond-ring
055.dice
056.dog
057.dolphin-101
058.doorknob
059.drinking-straw
060.duck
061.dumb-bell
062.eiffel-tower
063.electric-guitar-101
064.elephant-101
065.elk
066.ewer-101
067.eyeglasses
068.fern
069.fighter-jet
070.fire-extinguisher
071.fire-hydrant
072.

In [97]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [98]:
gaussian(X_train, y_train, X_test, y_test)

                               precision    recall  f1-score   support

                     001.ak47       0.00      0.00      0.00        20
            002.american-flag       0.00      0.00      0.00        19
                 003.backpack       0.09      0.23      0.13        30
             004.baseball-bat       0.00      0.00      0.00        25
           005.baseball-glove       0.06      0.07      0.06        30
          006.basketball-hoop       0.00      0.00      0.00        18
                      007.bat       0.00      0.00      0.00        21
                  008.bathtub       0.00      0.00      0.00        47
                     009.bear       0.00      0.00      0.00        20
                 010.beer-mug       0.00      0.00      0.00        19
                011.billiards       0.00      0.00      0.00        56
               012.binoculars       0.09      0.07      0.08        43
                 013.birdbath       0.00      0.00      0.00        20
     

C:\Users\Sebastiao\Desktop\Projetos\projeto-ic-automatocelular\projeto-ic-automatocelular\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sebastiao\Desktop\Projetos\projeto-ic-automatocelular\projeto-ic-automatocelular\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sebastiao\Desktop\Projetos\projeto-ic-automatocelular\projeto-ic-automatocelular\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in label

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em escala de cinza:

### Gerando histogramas

In [ ]:
#gerador_histogramas(dataset_1_dir, 128)

### Testando modelos de aprendizado

In [ ]:
histogramas, rotulos = retorna_dados(images_1_dir)
X = np.array(histogramas)
y = np.array(rotulos)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

# Dataset_2 CIFAR 100

## Em escala de cinza:

### Gerando histogramas

In [ ]:
#gerador_histogramas(dataset_2_dir, 32)

### Testando modelos de aprendizado

In [ ]:
histogramas, rotulos = retorna_dados(images_2_dir)
X = np.array(histogramas)
y = np.array(rotulos)

#Dividindo treino e teste na respectiva proporção (0.8 / 0.2), stratify = y mantém uma proporção entre os dados de teste de cada classe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em RGB:

### Gerando histogramas

In [ ]:
gerador_histogramas_RGB(dataset_2_dir, 32) #escala em 32 pq o dataset vem em 32x32

### Testando modelos

In [ ]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_2_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)